In [1]:
#Author Eren Ali Aslangiray
#14.06.2019

import joblib
import pandas as pd
import librosa
import pywt
import math
import numpy as np

In [3]:
X = joblib.load("X_Updated_joblib7k")
y = joblib.load("y_Updated_joblib7k")
X.pop(5519),y.pop(5519)

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 3)

In [85]:
adana = [1,2,3,4,5,6,7]
adana[-1]

7

In [34]:
mfcc = []
for item in X:
    mfccs = np.mean(librosa.feature.mfcc(y=item, sr=22050, n_mfcc=128).T,axis=0)
    mfcc.append(mfccs)

In [44]:
k = []
for item in X: 
    k.append(librosa.feature.spectral_bandwidth(item)[0])

In [96]:
finallist = []
medlist = []
counter1 = 0
for item in k:
    counter1 = counter1 + 1

    if len(item)<128:
        medlist = []
        diff = 128 - len(item)
        coppy = len(item) / diff
        if int(coppy)==0:
            coppy = math.ceil(coppy)
        if len(item)<64:
            for i in range (len(item)):
                if i % int(coppy) == 0:
                    medlist.append(item[i])
                    medlist.append(item[i])
                else:
                    medlist.append(item[i])
            diffin = 128-len(medlist)
            for i in range (diffin):
                medlist.append(item[-1])
                if len(medlist) == 128:
                    finallist.append(medlist)
                    medlist = []
                    break
            continue
        for i in range (len(item)):
            if i % int(coppy) == 0:
                medlist.append(item[i])
                medlist.append(item[i])
            else:
                medlist.append(item[i])            
            if len(medlist) == 128:
                finallist.append(medlist)
                medlist = []
                break
    elif len(item) == 128:
        finallist.append(item)
        continue
    else:
        diff = len(item) - 128
        remove = len(item) / diff
        ii = 0
        medlist = []
        for i in range (len(item)-1):
            if ii % math.ceil(remove) == 0:
                avgitem = (item[ii-1] + item[ii])/2
                medlist.append(avgitem)
                ii = ii + 2
            else:
                medlist.append(item[ii])
                ii = ii + 1
            if len(medlist) == 128:
                finallist.append(medlist)
                medlist = []
                break
print (counter1,len(finallist))

6996 6996


In [35]:
sr = 22050
X_db = []
for item in X:
    S = librosa.feature.melspectrogram(y=item, sr=sr)
    k = np.mean(librosa.amplitude_to_db(S, ref=np.max).T,axis=0)
    X_db.append(k)

In [36]:
waveletted = []
for item in X:
    a,b = pywt.dwt(item, 'db1')
    waveletted.append(a)

In [37]:
X2_new = []
a_new= []
for i in range(len(waveletted)):
    for item in waveletted[i]:
        if item > 0.001:
            a_new.append(item)
    X2_new.append(a_new)
    a_new = []

In [38]:
sample_final = []
sampled_list = []
for i in range(len(X2_new)):
    k = 0
    sums = 0
    for item in X2_new[i]:
        if len(sampled_list) == 128:
            continue
        sums = item + sums
        k = k+1
        if k == int(len(X2_new[i])/128):
            sampled_list.append(sums/k)
            k=0
            sums=0
    sample_final.append(sampled_list)
    sampled_list=[]

In [88]:
len(k),len(finallist),len(X_db),len(sample_final)

(6996, 6252, 6996, 6996)

In [98]:
x = []
for i in range(len(X)):
    x_t = []
    x_t.append(finallist[i])
    x_t.append(X_db[i])
    x_t.append(sample_final[i])
    x_t.append(mfcc[i])
    x.append(x_t)

In [99]:
y = np.asarray(y)
y2 = np.zeros((y.shape[0], 4),dtype=np.float32)
y2[np.arange(y.shape[0]), y] = 1.0

In [145]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(    
    np.asarray(adana), y2, test_size=0.20, random_state=42) 

In [129]:
from keras.models import Sequential
from keras.layers import Dense ,LSTM, Dropout
import numpy as np
from keras.callbacks import EarlyStopping,ModelCheckpoint
import keras

In [149]:
model = Sequential()
model.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25, input_shape=(4, 128)))
model.add(Dense(64,activation="relu"))
#model.add(Dropout(0.25))
model.add(Dense(32,activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dense(8,activation="relu"))
model.add(Dense(4, activation='softmax'))

earlystopper = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [150]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=64,
          verbose=0,callbacks=[earlystopper,checkpointer],epochs=1000)
model.load_weights('best_weights.hdf5')

Epoch 00103: early stopping


In [151]:
from sklearn import metrics
pred = model.predict(x_test)
pred2 = np.argmax(pred,axis=1)
y_compare = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_compare, pred2)
score2 = metrics.log_loss(y_compare, pred)
print("Final accuracy: {}".format(score))
print("Final log_loss: {}".format(score2))

Final accuracy: 0.8021428571428572
Final log_loss: 0.4630882895983926
